In [47]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

In [86]:
import numpy as np
from sklearn.model_selection import train_test_split

def generate_data(num_samples=10000, seq_length=5):
    """
    Generate a dataset consisting of two types of sequences:
    - Reversed sequences (labeled as 1)
    - Shuffled sequences (labeled as 0)
    """
    
    x_data_part1 = np.random.randint(1, 10, (num_samples // 2, seq_length))  # Generate random integer sequences
    x_data_part2 = np.random.randint(1, 10, (num_samples // 2, seq_length))  

    x_data_part1 = np.concatenate([x_data_part1, np.flip(x_data_part1, axis=1).copy()], axis=1)  # Concatenate with reversed version
    y_data_part1 = np.ones(num_samples // 2)  # Label reversed sequences as 1

    shuffled_x = np.array([np.random.permutation(row) for row in x_data_part2])
    x_data_part2 = np.concatenate([x_data_part2, shuffled_x], axis=1)  # Concatenate with shuffled version
    y_data_part2 = np.zeros(num_samples // 2)  # Label shuffled sequences as 0

    # Combine the two parts
    x_data = np.concatenate([x_data_part1, x_data_part2], axis=0)
    y_data = np.concatenate([y_data_part1, y_data_part2], axis=0)

    # Shuffle the dataset
    shuffle_indices = np.random.permutation(len(x_data))
    x_data = x_data[shuffle_indices]
    y_data = y_data[shuffle_indices]

    return x_data, y_data

# Generate dataset
x_data, y_data = generate_data(num_samples=10000)

# Split data into train (80%) and test (20%) sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)


In [87]:
# Train logistic regression model
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

# Predict on test set
y_pred_log_reg = log_reg.predict(x_test)

# Compute accuracy for logistic regression
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)

print(accuracy_log_reg)

0.494


d:\anaconda\envs\ml\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-10  # Avoid log(0)
    return -np.mean(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Derivative of sigmoid function
def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

# Neural network class with training and prediction
class SimpleNN:
    def __init__(self, input_size, hidden_size=20, learning_rate=0.01, epochs=100):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.start_learning_rate = learning_rate
        self.epochs = epochs

        # Initialize weights and biases
        self.W1 = np.random.randn(input_size, hidden_size) * 0.1
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, 1) * 0.1
        self.b2 = np.zeros((1, 1))

    def forward(self, X):
        """Forward propagation"""
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = sigmoid(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = sigmoid(self.Z2)
        return self.A2

    def backward(self, X, y):
        """Backward propagation and gradient update"""
        num_samples = X.shape[0]

        # Compute error
        dZ2 = self.A2 - y.reshape(-1, 1)
        dW2 = np.dot(self.A1.T, dZ2) / num_samples
        db2 = np.sum(dZ2, axis=0, keepdims=True) / num_samples

        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * sigmoid_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / num_samples
        db1 = np.sum(dZ1, axis=0, keepdims=True) / num_samples

        # Update weights and biases
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2

    def train(self, X_train, y_train, batch_size=4):
        """Train the neural network using mini-batch gradient descent"""
        num_samples = X_train.shape[0]
        num_batches = num_samples // batch_size
        losses = []

        for epoch in range(self.epochs):

            # cosine decrease
            self.learning_rate = 0.5 * (self.start_learning_rate) * (1 + np.cos(epoch / self.epochs * np.pi))

            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            X_train = X_train[indices]
            y_train = y_train[indices]

            epoch_loss = 0 
            
            for i in range(num_batches):

                start = i * batch_size
                end = start + batch_size
                X_batch = X_train[start:end]
                y_batch = y_train[start:end]

                y_pred = self.forward(X_batch)

                loss = binary_cross_entropy(y_batch, y_pred)
                epoch_loss += loss

                self.backward(X_batch, y_batch)

            if epoch % 5 == 0:
                avg_loss = epoch_loss / num_batches
                losses.append(avg_loss)
                print(f"Epoch {epoch}, Loss: {avg_loss:.4f}, Learning Rate: {self.learning_rate:.6f}")

    def predict(self, X):
        """Predict class labels"""
        y_pred = self.forward(X)
        return (y_pred >= 0.5).astype(int)

# Example usage
if __name__ == "__main__":

    # Initialize and train the neural network
    nn = SimpleNN(input_size=10, hidden_size=10, learning_rate=0.05, epochs=100)
    nn.train(x_train, y_train)

    # Make predictions
    y_pred_nn = nn.predict(x_test)

    # Compute accuracy
    accuracy_nn = accuracy_score(y_test, y_pred_nn)
    print(f"Neural Network Accuracy: {accuracy_nn:.4f}")


Epoch 0, Loss: 0.6476, Learning Rate: 0.050000
Epoch 5, Loss: 0.1445, Learning Rate: 0.049692
Epoch 10, Loss: 0.1120, Learning Rate: 0.048776
Epoch 15, Loss: 0.0964, Learning Rate: 0.047275
Epoch 20, Loss: 0.0886, Learning Rate: 0.045225
Epoch 25, Loss: 0.0827, Learning Rate: 0.042678
Epoch 30, Loss: 0.0752, Learning Rate: 0.039695
Epoch 35, Loss: 0.0624, Learning Rate: 0.036350
Epoch 40, Loss: 0.0618, Learning Rate: 0.032725
Epoch 45, Loss: 0.0599, Learning Rate: 0.028911
Epoch 50, Loss: 0.0578, Learning Rate: 0.025000
Epoch 55, Loss: 0.0545, Learning Rate: 0.021089
Epoch 60, Loss: 0.0540, Learning Rate: 0.017275
Epoch 65, Loss: 0.0530, Learning Rate: 0.013650
Epoch 70, Loss: 0.0529, Learning Rate: 0.010305
Epoch 75, Loss: 0.0524, Learning Rate: 0.007322
Epoch 80, Loss: 0.0520, Learning Rate: 0.004775
Epoch 85, Loss: 0.0518, Learning Rate: 0.002725
Epoch 90, Loss: 0.0515, Learning Rate: 0.001224
Epoch 95, Loss: 0.0514, Learning Rate: 0.000308
Neural Network Accuracy: 0.9875
